# Dusty box analysis

This notebook contains analysis of the dusty box test for multigrain dust.

## Imports

In [ ]:
import importlib
import pathlib
from pathlib import Path
from typing import Any, Dict, List, Tuple

In [ ]:
import numpy as np
import pandas as pd
import plonk
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import all_palettes
from bokeh.plotting import figure
output_notebook()

## Functions

Load dustybox analysis functions:
- `generate_results`: to perform the analysis

In [ ]:
loader = importlib.machinery.SourceFileLoader('dustybox', '../modules/dustybox/analysis.py')
dustybox = loader.load_module()

## Path to data

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustybox').expanduser()
paths = sorted(list(root_directory.glob('*')))

## Perform analysis

In [ ]:
dataframes = dict()

for p in paths:
    sim = plonk.load_sim(prefix='dustybox', directory=p)
    name = '-'.join(sim.path.name.split('=')[-1].split('-')[::-1])
    print(f'Running analysis on {name}')
    dataframes[name] = dustybox.generate_results(sim)
    del sim

Inspect one data frame:

In [ ]:
dataframes['MRN-0.01']

## Plot results

Define function to plot results from one simulation.

In [ ]:
def plot(name, df):
    """Plot one simulation."""
    n_dust = int((len(df.columns) - 1) / 3)
    palette = all_palettes['Viridis'][n_dust]

    x = [df['time'] for col in df.columns if col.startswith('data')]
    y_data = [df[col] for col in df.columns if col.startswith('data')]
    y_exact1 = [df[col] for col in df.columns if col.startswith('exact1')]
    y_exact2 = [df[col] for col in df.columns if col.startswith('exact2')]

    fig = figure(title=name)
    fig.multi_line(x, y_exact1, line_dash='solid', line_color=palette, line_width=2)
    fig.multi_line(x, y_exact2, line_dash=[10, 10], line_color=palette, line_width=2)
    for xx, yy, color in zip(x, y_data, palette):
        fig.scatter(xx, yy, line_color=color, fill_color=None, size=6)

    return fig

Plot all simulation results.

In [ ]:
ncols = 2
figs = list()

for name, df in dataframes.items():
    fig = plot(name, df)
    figs.append(fig)
    
p = gridplot(figs, ncols=ncols, sizing_mode='stretch_width', plot_height=300)
show(p)